In [1]:
!mkdir outputs

In [2]:
%%writefile model.py
import torchvision.models as models
import torch.nn as nn

def build_model(pretrained=True, fine_tune=True):
    if pretrained:
        print('[INFO]: Loading Pretrained Weights')
    elif not pretrained:
        print('[INFO]: Not Loading Pretrained Weights')
    model = models.shufflenet_v2_x1_0(pretrained=pretrained)
    
    if fine_tune:
        print('[INFO]: Fine-Tuning All Layers')
        for param in model.parameters():
            param.requires_grad = True
    elif not fine_tune:
        print('[INFO]: Freezing Hidden Layers')
        for param in model.parameters():
            param.requires_grad = False
    
    model.fc = nn.Linear(1024, 38)
    return model

Writing model.py


In [3]:
!python model.py

In [4]:
%%writefile datasets.py
import torch

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

valid_split = 0.2
seed = 42
batch_size = 64
root_dir = '/kaggle/input/new-plant-diseases-dataset'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = datasets.ImageFolder(root_dir, transform=transform)

datasetsize = len(dataset)
print('The total no.of images is:', datasetsize)

valid_size = int(valid_split*datasetsize)
train_size = len(dataset) - valid_size

train_data, valid_data = torch.utils.data.random_split(
    dataset, [train_size, valid_size]
)

print(f"Total training images: {len(train_data)}")
print(f"Total valid images: {len(valid_data)}")

train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, num_workers=4
)
valid_loader = DataLoader(
    valid_data, batch_size=batch_size, shuffle=True, num_workers=4
)

Writing datasets.py


In [5]:
!python datasets.py

The total no.of images is: 175767
Total training images: 140614
Total valid images: 35153


In [6]:
%%writefile utils.py
import torch
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

def save_model(epochs, model, optimizer, criterion):
    torch.save({
        'epoch': epochs,
        'model_statae_dict': model.state_dict(),
        'loss': criterion,
    }, 'outputs/model.pth')
    
def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='green', linestyle='-',
        label='train_accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-',
        label='validation accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')
    
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='orange', linestyle='-',
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-',
        label='validation loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')

Writing utils.py


In [7]:
%%writefile train.py
import torch
import argparse
import torch.nn as nn
import torch.optim as optim

from model import build_model
from utils import save_model, save_plots
from datasets import train_loader, valid_loader
from tqdm.auto import tqdm

parser = argparse.ArgumentParser()
parser.add_argument('-e', '--epochs', type=int, default=20,
                   help='number of epochs to train our network for')
args = vars(parser.parse_args())

lr = 0.001
epochs = args['epochs']
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Computation device: {device}\n')
model = build_model(pretrained=True, fine_tune=False).to(device)

total_params = sum(p.numel() for p in model.parameters())

print(f"{total_params:,} total parameters.")

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

optimizer = optim.Adam(model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds ==  labels).sum().item()
        loss.backward()
        optimizer.step()
    epoch_loss = train_running_loss / counter
    epoch_acc = 100 * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

def validate(model, testloader, criterion):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            outputs = model(image)
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100 *  (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

train_loss, valid_loss = [], []
train_acc, valid_acc = [], []

for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch + 1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader,
                                             optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,
                                                criterion)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training Loss: {train_epoch_loss:.3f}, training_acc: {train_epoch_acc:.3f}")
    print(f"Validation Loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('$'*50)
    
save_model(epochs, model, optimizer, criterion)
save_plots(train_acc, valid_acc, train_loss, valid_loss)
print('Training Complete')

Writing train.py


In [ ]:
!python train.py --epochs 100

The total no.of images is: 175767
Total training images: 140614
Total valid images: 35153
Computation device: cuda

[INFO]: Loading Pretrained Weights
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100%|██████████████████████████████████████| 8.79M/8.79